In [2]:
import torch.backends.cudnn as cudnn


In [1]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\KizorSelvaraj\AppData\Roaming\nltk_data.
[nltk_data]    |     ..
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\KizorSelvaraj\AppData\Roaming\nltk_data.
[nltk_data]    |     ..
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\KizorSelvaraj\AppData\Roaming\nltk_data.
[nltk_data]    |     ..
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\KizorSelvaraj\AppData\Roaming\nltk_data.
[nltk_data]    |     ..
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | D

True

In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
#importing the important package for the model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk import WordNetLemmatizer,word_tokenize,PorterStemmer
from nltk.corpus import stopwords
import sklearn 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [4]:
#Reading the Raw data from spotify datset 
rwdata=pd.read_csv("spotify_millsongdata.csv")

Understanding the values in dataset like - Null values,Duplicate values,unique value and shape

In [5]:
#Info dataset 
rwdata.info()


#There is 4 columns and all are in string type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [6]:
#Find the shape of the dataset
print(f'Row :',rwdata.shape[0])
print(f'Columns :',rwdata.shape[1])


Row : 57650
Columns : 4


In [7]:
#Finding the null values
rwdata.isnull().sum().sort_values(ascending=False)

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
#Finding the Unique values
rwdata.nunique().sort_values(ascending=False)

link      57650
text      57494
song      44824
artist      643
dtype: int64

In [9]:
#Finding the duplicate values 
rwdata.duplicated().sum()

0

In [10]:
#droping the unwanted columns
rwdata=rwdata.drop(columns=['link'],axis=1).reset_index(drop=True)

In [11]:
rwdata['artist']

0                ABBA
1                ABBA
2                ABBA
3                ABBA
4                ABBA
             ...     
57645    Ziggy Marley
57646    Ziggy Marley
57647            Zwan
57648            Zwan
57649            Zwan
Name: artist, Length: 57650, dtype: object

In [12]:
#As dataset has 57650 data. Let make it to smaller to get faster output
spotiy=rwdata.sample(3892)

I have save the dataset to my localdisk for my further reference

In [13]:
spotiy.to_csv("spotify.csv",index=False)

In [14]:
# import the csv file # It also change the index to 0-9
spotiy=pd.read_csv('spotify.csv')

In [15]:
spotiy

,artist,song,text
0,Eminem,Crazy In Love,Tell myself that I was doin' all right \r\nTh...
1,Matt Redman,The Father's Song,I have heard so many songs \r\nListened to a ...
2,Ramones,Outsider,I'm an outsider outside of everything \r\nI'm...
3,Nazareth,Backroom Boys,The backroom boys are playin' real hard \r\nT...
4,Tim McGraw,Don't Take The Girl,Johnny's daddy was taking him fishin' \r\nWhe...
...,...,...,...
3887,Reba Mcentire,Is There Life Out There,She married when she was twenty \r\nShe thoug...
3888,Elvis Presley,Edge Of Reality,I walk along a thin line darling \r\nDark sha...
3889,Ellie Goulding,Every Time You Go,Every time you go \r\nEvery time you go \r\n...
3890,Extreme,Fair Weather Faith,Prentending to act so serene \r\nOn your so c...


In [16]:
spotiy.text


0       Tell myself that I was doin' all right  \r\nTh...
1       I have heard so many songs  \r\nListened to a ...
2       I'm an outsider outside of everything  \r\nI'm...
3       The backroom boys are playin' real hard  \r\nT...
4       Johnny's daddy was taking him fishin'  \r\nWhe...
                              ...                        
3887    She married when she was twenty  \r\nShe thoug...
3888    I walk along a thin line darling  \r\nDark sha...
3889    Every time you go  \r\nEvery time you go  \r\n...
3890    Prentending to act so serene  \r\nOn your so c...
3891    The corpses all hang headless and limp  \r\nBo...
Name: text, Length: 3892, dtype: object

In [17]:
spotiy['text']=spotiy['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)

Stemming for text - which is used to normalize the word in the text

In [18]:
stem=PorterStemmer()

def token(text):
    token=word_tokenize(text)
    stemword=[stem.stem(i) for i in token]
    return " ".join(stemword)    

In [19]:
# use lambda funcation to apply this funcation in all the text dataset
spotiy['text']=spotiy['text'].apply(lambda x: token(x))

Now we are going to convert the word into numbers. Word to vectorizer

myself i use TF-IDF.

We also has Keras one-hot encoding,BoW,Word2vec

In [20]:
# Fitting the text into vectorizer 
vector=TfidfVectorizer(analyzer="word",stop_words="english")
t2n=vector.fit_transform(spotiy['text'])

In [21]:
spotiy['text'][300]

"how fast can i run ? how far can i go ? these are the question , the young want to know i met her in brooklyn , she wa come from a place i said `` where are you go to '' ? she said `` where have you been '' ? 'caus i 'm trap in here , set me free i know your world let me see take me down that last open road take me down that last open road take me down that last open road i know there is so much past thi place i see it smile on your jag , rag face someth tell me , someth strong there 's a world i 've heard of , so befor it 's gone take me down that last open road take me down that last open road"

Cosine similarity is use to find the similary between 2 varible

In [22]:
# Fit in to cosine which is used to find the similarity between 2 variable 
songsim=cosine_similarity(t2n)
songsim[0]

array([1.        , 0.00660763, 0.00353751, ..., 0.08069979, 0.03317859,
       0.03359381])

In [23]:
spotiy['song']

0                 Crazy In Love
1             The Father's Song
2                      Outsider
3                 Backroom Boys
4           Don't Take The Girl
                 ...           
3887    Is There Life Out There
3888            Edge Of Reality
3889          Every Time You Go
3890         Fair Weather Faith
3891                     Skulls
Name: song, Length: 3892, dtype: object

In [24]:
from sympy import re


d=sorted(list(enumerate(songsim[2020])),reverse=True,key=lambda x:x[1])
d[0:5]
song=[]
for i in d[0:5]:
    song.append(spotiy.iloc[i[0]].song)

song

['Boku Wa Kuma',
 "Feelin' On Yo Booty",
 'This Was My Life',
 "I Wasn't Fooling Around",
 "I Don't Believe You've Met My Baby"]

In [27]:
#Example #spotiy[spotiy['song']=='Feelin']
#spotiy[spotiy['song']=='Farmer John'].index[0]
def recommander(song_name):
    ids=spotiy[spotiy['song']==song_name].index[0]
    dis=sorted(list(enumerate(songsim[ids])),reverse=True,key=lambda x:x[1])  
    song=[]
    art=[]
    for i in dis[1:6]:
        s=spotiy.iloc[i[0]].song
        a=spotiy.iloc[i[0]].artist
        song.append(s)
        art.append(spotiy.iloc[i[0]].artist)
    return song,art

In [30]:
recommander("Boku Wa Kuma")

(["Feelin' On Yo Booty",
  'This Was My Life',
  "I Wasn't Fooling Around",
  "I Don't Believe You've Met My Baby",
  'Trashed'],
 ['R. Kelly', 'Megadeth', 'George Strait', 'Alison Krauss', 'Black Sabbath'])

save the file as pickle to run in streamlit

In [31]:
pickle.dump(songsim,open("music_recommendation.pkl","wb"))
pickle.dump(spotiy,open("df_music.pkl","wb"))